tidied up version of convex mesh generation code

In [1]:
from skimage import io, data
from skimage import filters, measure
import napari
from scipy import ndimage as ndi
import numpy as np
from PIL import Image
from aicsimageio import AICSImage, imread

ImportError: cannot import name 'diff_treestructure' from 'xarray.core.formatting' (C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\xarray\core\formatting.py)

In [2]:
from scipy.spatial import ConvexHull, Delaunay

In [3]:
viewer = napari.Viewer(title='convex_hull_surface')

In [ ]:
image_1_name=str('i_ch1')
layer_1_name=image_1_name
image_2_name=str('i_ch2')
layer_2_name=image_2_name
file_path=str('file_path')

In [4]:
image_i = AICSImage(file_path)
print(image_i.shape)

(20, 2, 80, 485, 623)


In [5]:
i_ch1 = image_i.data[:,0,:,:]
i_ch2 = image_i.data[:,1,:,:]


In [6]:
viewer.add_image(i_ch1, name='i_ch1',colormap='magenta',blending='additive')
viewer.layers['i_ch1'].scale=[6,1,1]
viewer.add_image(i_ch2, name='i_ch2',colormap='green',blending='additive')
viewer.layers['i_ch2'].scale=[6,1,1]


### convex hull generation

In [31]:
convex_points=viewer.layers['Points'].data

In [12]:
relevant_indices=[1,2,3]
convex_points_3D = [convex_points[i] for i in relevant_indices]

In [36]:
array_3D = []
for a in range(0,len(convex_points)):
    array_3D_element = [convex_points[a][i] for i in relevant_indices]
    print(array_3D_element)
    array_3D.append(array_3D_element)

[56.0, 312.64481915157353, 140.41066274480036]
[56.0, 291.1583765547404, 192.49900843409276]
[56.0, 327.6202185372451, 202.91667757195123]
[61.0, 323.062488289432, 154.0838534882396]
[61.0, 296.3672111236697, 141.06176706591654]
[61.0, 288.5539592702758, 209.4277207831128]
[61.0, 336.73567903287125, 217.24097263650665]


In [38]:
cell_hull=ConvexHull(array_3D)

In [39]:
cell_convex_test=cell_hull.simplices #generate vertices of convex hull

In [41]:
surface=viewer.add_surface(data=(convex_points,cell_convex_test))
surface.normals.face.visible = True
surface.normals.vertex.visible = True
surface.wireframe.visible = True

In [42]:
viewer.layers['Surface'].scale=[6,1,1]